In [2]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from ipywidgets import Checkbox, HBox, VBox, Label, Dropdown, Button, Output
from tqdm import tqdm

import numpy as np
import requests
from shapely.ops import transform
from scipy.interpolate import Rbf

# ------------------------------------------------------------------
# Data definitions (same as before)
# ------------------------------------------------------------------
cities = {
    'Brussels': {'path': '../../data/osm/raw/shapefiles/Brussels', 'bbox': (4.305, 50.805, 4.425, 50.875)},  # (minx, miny, maxx, maxy)
    'Maastricht': {'path': '../../data/osm/raw/shapefiles/Maastricht', 'bbox': (5.66, 50.837, 5.72, 50.86)},
    'Europe': {'path': '../../data/osm/raw/shapefiles/Europe', 'bbox': (-10, 36, 30, 70)},
}

data_files = {
    'Roads': 'gis_osm_roads_free_1.shp',
    'Buildings': 'gis_osm_buildings_a_free_1.shp',
    'Waterways': 'gis_osm_waterways_free_1.shp',
    'Landuse': 'gis_osm_landuse_a_free_1.shp',
    'Natural': 'gis_osm_natural_free_1.shp',
    'Places': 'gis_osm_places_free_1.shp',
    'POIs': 'gis_osm_pois_free_1.shp',
    'Railways': 'gis_osm_railways_free_1.shp',
    'Traffic': 'gis_osm_traffic_free_1.shp',
    'Transport': 'gis_osm_transport_free_1.shp',
    'Water': 'gis_osm_water_a_free_1.shp',
    'Countries': 'ne_10m_admin_0_countries.shp'
}

layer_params = {
    'Landuse': {'color': 'palegreen', 'alpha': 0.5},
    'Waterways': {'color': 'deepskyblue', 'linewidth': 1},
    'Buildings': {'color': 'gray', 'alpha': 0.7},
    'Roads': {'color': 'black', 'linewidth': 0.5},
    'Natural': {'color': 'green', 'alpha': 0.5},
    'Places': {'color': 'orange', 'alpha': 0.5},
    'POIs': {'color': 'purple', 'alpha': 0.5},
    'Railways': {'color': 'brown', 'linewidth': 1},
    'Traffic': {'color': 'red', 'alpha': 0.5},
    'Transport': {'color': 'blue', 'alpha': 0.5},
    'Water': {'color': 'aqua', 'alpha': 0.5},
    'Countries': {'color': 'lightgray', 'alpha': 0.5},
}

# ------------------------------------------------------------------
# Global variables and widgets (same as before)
# ------------------------------------------------------------------
map_data = {}    # cache for loaded layers
current_fig = None
current_ax = None

map_out = Output()
info_label = Label(value="")

city_dropdown = Dropdown(
    options=list(cities.keys()),
    value='Maastricht',
    description='City:'
)

layer_checkboxes = {}
for layer_name in data_files.keys():
    cb = Checkbox(value=(layer_name == 'Roads'),
                  description=layer_name)
    layer_checkboxes[layer_name] = cb


controls_box = VBox([city_dropdown, info_label] + list(layer_checkboxes.values()))
dashboard = HBox([map_out, controls_box])
display(dashboard)

# ------------------------------------------------------------------
# Helper functions (same as before)
# ------------------------------------------------------------------
def get_file_size_label(city, layer_name):
    file_path = os.path.join(cities[city]['path'], data_files[layer_name])
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path) / (1024 * 1024)
        return f"{layer_name} ({file_size:.2f} MB)"
    else:
        return f"{layer_name} (not found)"

def update_checkboxes_labels():
    city = city_dropdown.value
    for layer_name, cb in layer_checkboxes.items():
        cb.description = get_file_size_label(city, layer_name)

def load_and_filter_data(layer_name, city):
    if layer_name not in map_data:
        info_label.value = f"Loading {layer_name} data..."
        shp_path = os.path.join(cities[city]['path'], data_files[layer_name])
        try:
            map_data[layer_name] = gpd.read_file(shp_path, bbox=cities[city]['bbox'])
        except Exception as e:
            print(f"Error loading {layer_name}: {e}")
            map_data[layer_name] = None
        info_label.value = ""
    return map_data[layer_name]

def update_map(*args):
    global current_fig, current_ax, map_data
    city = city_dropdown.value

    fig, ax = plt.subplots(1, figsize=(10, 14))
    bbox = cities[city]['bbox']
    ax.set_xlim(bbox[0], bbox[2])
    ax.set_ylim(bbox[1], bbox[3])
    ax.set_aspect('equal')
    ax.set_title(city)

    for layer_name, cb in layer_checkboxes.items():
        if cb.value:
            info_label.value = f"Adding {layer_name}..."
            data = load_and_filter_data(layer_name, city)
            if data is not None and not data.empty:
                params = layer_params.get(layer_name, {})
                data.plot(ax=ax, **params)
                if layer_name == 'Places':
                    for idx, row in data.iterrows():
                        try:
                            name = row.get('name', '')
                            if name:
                                ax.text(row.geometry.x, row.geometry.y, name, fontsize=10,
                                        ha='center', color='red', weight='bold',
                                        bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))
                        except Exception as e:
                            print(f"Error adding label for Places: {e}")
    info_label.value = ""
    current_fig = fig
    current_ax = ax

    with map_out:
        map_out.clear_output(wait=True)
        plt.show()

# ------------------------------------------------------------------
# Event handlers and observers (same as before)
# ------------------------------------------------------------------
def on_city_change(change):
    global map_data
    if change['type'] == 'change' and change['name'] == 'value':
        map_data = {}  # clear cached data when city changes
        update_checkboxes_labels()
        update_map()

city_dropdown.observe(on_city_change)

for cb in layer_checkboxes.values():
    cb.observe(lambda change: update_map(), names='value')


# ------------------------------------------------------------------
# Initial drawing of the map
# ------------------------------------------------------------------
update_checkboxes_labels()
update_map()

Error loading Roads: ../../data/osm/raw/shapefiles/Europe/gis_osm_roads_free_1.shp: No such file or directory


Error loading Countries: ../../data/osm/raw/shapefiles/Maastricht/ne_10m_admin_0_countries.shp: No such file or directory


In [2]:
def get_osrm_duration(p1, p2, profile='foot'):
    base_url = f"http://localhost:5000/route/v1/{profile}/"
    coords = f"{p1[0]},{p1[1]};{p2[0]},{p2[1]}"
    url = f"{base_url}{coords}?overview=false"
    try:
        response = requests.get(url)
        data = response.json()
        if 'routes' in data and len(data['routes']) > 0:
            duration = data['routes'][0]['duration']
            return duration
        else:
            print("OSRM query failed:", data)
            return None
    except Exception as e:
        print("OSRM query failed:", e)
        return None